The purpose of the following code was to prepare the videos' metadata, which was generated in notebook 2 (Scraping YouTube metadata), for later analyses and calculations. Parts of this code were generated with the help of ChatGPT and altered for the specific needs of this study.

In [2]:
import pandas as pd

BILD = pd.read_csv("BILD_video_data_cleaned.csv")
DW = pd.read_csv("DW_video_data_cleaned.csv")

print(BILD.head())
print(DW.head())


      video_id                                    video_url  \
0  Jf7BcLknC1I  https://www.youtube.com/watch?v=Jf7BcLknC1I   
1  KFtEWl4s-sk  https://www.youtube.com/watch?v=KFtEWl4s-sk   
2  k5wLZkD9NZM  https://www.youtube.com/watch?v=k5wLZkD9NZM   
3  3j1EFKPSGoE  https://www.youtube.com/watch?v=3j1EFKPSGoE   
4  QkGZYUbNnGE  https://www.youtube.com/watch?v=QkGZYUbNnGE   

                                               title  \
0  Leopard und Puma im Einsatz: Bundeswehr probt ...   
1  Jagd auf jüdische Passagiere: Islamisten-Mob s...   
2  Iran gibt Huthis Angriffsbefehl auf Israel | B...   
3  Paul Ronzheimer mitten im Kampfgebiet: Heftige...   
4  Ukraine kämpft um ihren Brückenkopf | BILD-Lag...   

                                         description  \
0  Hier schießen der Leopard und der Puma-Panzer ...   
1  Erschütternde Szenen an einem Flughafen in der...   
2  Die De-Facto-Regierung des Jemen hat Israel am...   
3  Maschinengewehrfeuer, Kampfjets und Panzer: Pa...   
4  U

In [3]:
BILD["year"] = BILD["published_at"].str[:4].astype(int)
DW["year"] = DW["published_at"].str[:4].astype(int)

BILD = BILD[BILD["year"].between(2020, 2023)]
DW = DW[DW["year"].between(2020, 2023)]

BILD = BILD.drop(columns=["year"])
DW = DW.drop(columns=["year"])


In [4]:
#extract hashtags

import re

def extract_hashtags(text):
    if isinstance(text, str):
        return re.findall(r"#\w+", text)
    return []

BILD["hashtags"] = BILD["description"].apply(extract_hashtags)
DW["hashtags"] = DW["description"].apply(extract_hashtags)

print(BILD[["description", "hashtags"]].head())
print(DW[["description", "hashtags"]].head())


                                         description  \
0  Hier schießen der Leopard und der Puma-Panzer ...   
1  Erschütternde Szenen an einem Flughafen in der...   
2  Die De-Facto-Regierung des Jemen hat Israel am...   
3  Maschinengewehrfeuer, Kampfjets und Panzer: Pa...   
4  Ukrainische Soldaten versuchen mit aller Kraft...   

                                 hashtags  
0          [#Puma, #Leopard, #Bundeswehr]  
1        [#Israel, #Islamisten, #Angriff]  
2          [#Iran, #Israel, #Lagezentrum]  
3               [#israel, #gaza, #geisel]  
4  [#Ukraine, #Brückenkopf, #Lagezentrum]  
                                         description  \
0  Bundeskanzler Olaf Scholz hat sich in seiner N...   
1  Eine 14-jährige wird jahrelang vergewaltigt un...   
2  Kinder, die im Krieg gegen die Ukraine den Vat...   
3  Die Ukraine braucht mehr Soldaten - die Zeiten...   
4  Das James-Webb-Teleskop kann bis in die Frühze...   

                                          hashtags  
0  [#Deut

In [5]:
BILD_description_end_phrases = [
    "BILD IM TV",
    "Abonniert jetzt unseren Kanal und aktiviert die Glocke!",
    "JETZT Sendersuchlauf starten",
    "► Die ganze Sendung hier:"
]

DW_description_end_phrases = [
    "DW Deutsch Abonnieren"
]

In [6]:
def make_cut_function(phrases):
    pattern = r"|".join(map(re.escape, phrases))
    escaped_phrases = [re.escape(p) for p in phrases if p != "------------"]
    dash_regex = r"-{12,}"

    pattern_parts = escaped_phrases + [dash_regex]
    pattern = r"|".join(pattern_parts)

    def cut_description(text):
        if not isinstance(text, str):
            return text
        return re.split(pattern, text)[0].strip()

    return cut_description

In [7]:
cut_BILD = make_cut_function(BILD_description_end_phrases)
cut_DW = make_cut_function(DW_description_end_phrases)

BILD["description"] = BILD["description"].apply(cut_BILD)
DW["description"] = DW["description"].apply(cut_DW)

In [8]:
# concatenating title and description for future topic classification
BILD["text"] = BILD["title"].astype(str) + ": " + BILD["description"].astype(str)
DW["text"] = DW["title"].astype(str) + ": " + DW["description"].astype(str)

In [9]:
BILD["description"] = BILD["description"].apply(cut_BILD)
DW["description"] = DW["description"].apply(cut_DW)

In [10]:
BILD.to_csv("BILD_ready_for_classification.csv", index=False)
DW.to_csv("DW_ready_for_classification.csv", index=False)

from google.colab import files

files.download("BILD_ready_for_classification.csv")
files.download("DW_ready_for_classification.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>